In [1]:
# Install required packages
#%pip install --upgrade -q google-genai google-adk==1.9.0 a2a-sdk==0.3.0 python-dotenv aiohttp uvicorn requests mermaid-python nest-asyncio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.7.0 which is incompatible.
embedchain 0.1.113 requires chromadb<0.5.0,>=0.4.24, but you have chromadb 1.0.13 which is incompatible.
embedchain 0.1.113 requires langchain<0.2.0,>=0.1.4, but you have langchain 0.3.26 which is incompatible.
embedchain 0.1.113 requires langchain-cohere<0.2.0,>=0.1.4, but you have langchain-cohere 0.4.4 which is incompatible.
embedchain 0.1.113 requires langchain-openai<0.2.0,>=0.1.7, but you have langchain-openai 0.3.25 which is incompatible.
crewai 0.28.8 requires langchain<0.2.0,>=0.1.10, but you have langchain 0.3.26 which is incompatible.
crewai 0.28.8 requires python-dotenv==1.0.0, but you have python-dotenv 1.2.1 which is incompatible.
s3fs 2024.3.1 requires fsspec==2024.3.1, but you have fsspec 

In [2]:
import sys

from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver


class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore

In [3]:
import asyncio
import logging
import os
import sys
import threading
import time

from typing import Any

import httpx
import nest_asyncio
import uvicorn

from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

/opt/anaconda3/lib/python3.12/site-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [9]:
# Import API key
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Defining Agents

1. Router Agent
2. Customer Data Agent
3. Support Agent

In [ ]:
router_agent = Agent(
    model="gpt-4o-mini",
    name="router_agent",
    instruction="""
You are an Orchestration Agent. Your job is to break down a user query into a sequence of steps for specialist agents.

Available Agents:
1. Customer Data Agent: Can 'retrieve', 'list', or 'update' customer data. Access to database.
2. Support Agent: Can handle 'support' requests, create tickets, and assess priority.

Rules:
- If the user asks for help/support but hasn't provided data, you might need to get data first (e.g., list customers or get ID).
- If the user asks for a complex multi-step operation (e.g., "for all customers"), break it down if possible, or send a list query first.
- "pass_context": Set to true if the result of previous steps should be passed to this step.
- IMPORTANT: Always preserve customer IDs in queries to the Support Agent. If a customer ID is mentioned in the original query, include it in the Support Agent query.

Example Plans:

Scenario: "Get info for customer 5"
{
  "steps": [
    {
        "action": "call_data_agent", 
        "query": "Get info for customer 5", 
        "pass_context": false
    }
],
  "final_synthesis": "Present the data to the user."
}

Scenario: "I need help with my account, customer ID 5"
{
  "steps": 
  [
    {
        "action": "call_data_agent", 
        "query": "Get info for customer 5", 
        "pass_context": false
    },
    {
        "action": "call_support_agent", 
        "query": "I need help with my account, customer ID 5", "pass_context": true
    }
  ],
  "final_synthesis": "Summarize the support action taken."
}

Scenario: "I want to cancel my subscription but I'm having billing issues, customer ID 1"
{
  "steps": [
    {
        "action": "call_data_agent", 
        "query": "Get info for customer 1", 
        "pass_context": false
    },
    {
        "action": "call_support_agent", 
        "query": "I want to cancel my subscription and I'm having billing issues, 
        customer ID 1", "pass_context": tru
    }
  ],
  "final_synthesis": "Summarize the cancellation and billing support action taken."
}
"""
)

In [ ]:
orchestration_agent = Agent(
    
)


def create_orchestration_plan(self, query: str) -> Dict[str, Any]:
        """
        Create a dynamic orchestration plan using LLM.
        """
        system_prompt = """You are an Orchestration Agent. Your job is to break down a user query into a sequence of steps for specialist agents.

Available Agents:
1. Customer Data Agent: Can 'retrieve', 'list', or 'update' customer data. Access to database.
2. Support Agent: Can handle 'support' requests, create tickets, and assess priority.

Available Actions:
- "call_data_agent": Query the Customer Data Agent.
- "call_support_agent": Query the Support Agent.

Rules:
- If the user asks for help/support but hasn't provided data, you might need to get data first (e.g., list customers or get ID).
- If the user asks for a complex multi-step operation (e.g., "for all customers"), break it down if possible, or send a list query first.
- "pass_context": Set to true if the result of previous steps should be passed to this step.
- IMPORTANT: Always preserve customer IDs in queries to the Support Agent. If a customer ID is mentioned in the original query, include it in the Support Agent query.

Example Plans:

Scenario: "Get info for customer 5"
{
  "steps": [
    {"action": "call_data_agent", "query": "Get info for customer 5", "pass_context": false}
  ],
  "final_synthesis": "Present the data to the user."
}

Scenario: "I need help with my account, customer ID 5"
{
  "steps": [
    {"action": "call_data_agent", "query": "Get info for customer 5", "pass_context": false},
    {"action": "call_support_agent", "query": "I need help with my account, customer ID 5", "pass_context": true}
  ],
  "final_synthesis": "Summarize the support action taken."
}

Scenario: "I want to cancel my subscription but I'm having billing issues, customer ID 1"
{
  "steps": [
    {"action": "call_data_agent", "query": "Get info for customer 1", "pass_context": false},
    {"action": "call_support_agent", "query": "I want to cancel my subscription and I'm having billing issues, customer ID 1", "pass_context": true}
  ],
  "final_synthesis": "Summarize the cancellation and billing support action taken."
}

Scenario: "Status of tickets for all active customers"
{
  "steps": [
    {"action": "call_data_agent", "query": "List all active customers", "pass_context": false},
    {"action": "call_support_agent", "query": "Get ticket status for these customers", "pass_context": true}
  ],
  "final_synthesis": "Aggregated status report."
}

Respond ONLY with valid JSON.
"""